# Katalog B: Oberflächenfunde
(siehe Wotzka 1995: 391-410)

In [139]:
%matplotlib inline
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

con = sqlite3.connect("../data/CongoDB.sqlite")

In [140]:
sql = """SELECT
           t_Obj.objID, 
           't_Ort'.'ort_name' || ' (' || 't_Ort'.'ort_kurz' || '; Fpl. ' || 't_Ort'.'Kat-Nr' || ')' AS ORT, 
           't_Ort'.'Kat-Nr',
           't_Komplex'.'bef_nr' || ' (' || 't_Komplex'.'bef_beschr' || ')' AS BEF,
           t_Obj.Art,
           t_Obj.Scherbe,
           t_Obj.Anzahl,
           t_Obj.Typ           
       FROM (t_Ort INNER JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           INNER JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID
       WHERE (((t_Ort.ort_lit)='DS')
           AND ((t_Obj.Art) = "K")
           AND ((t_Komplex.bef_art) Like '%Ob%'))""".replace('\n',' ')

df = pd.read_sql(sql, con)
df['Anzahl'] = df['Anzahl'].convert_objects(convert_numeric=True)
df['Typ'] = df['Typ'].replace('', '?', regex=True)
display(df.head())

C:\Users\DirkSeidenticker\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:17: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


,objID,ORT,Kat-Nr,BEF,Art,Scherbe,Anzahl,Typ
0,10,Batanga (BAT; Fpl. 209),209,85/101 (),K,R,1,DAM (?)
1,11,Batanga (BAT; Fpl. 209),209,85/101 (),K,R,1,?
2,12,Batanga (BAT; Fpl. 209),209,85/101 (),K,R,1,Roulette
3,13,Batanga (BAT; Fpl. 209),209,85/101 (),K,W,1,Roulette
4,14,Batanga (BAT; Fpl. 209),209,85/101 (),K,W,1,?


In [141]:
df_pivot = df.pivot_table(values = 'Anzahl', index = ['Kat-Nr', 'ORT', 'BEF', 'Typ'], columns = 'Scherbe', aggfunc = sum, fill_value = '-', margins = True)
display(df_pivot.head())

Scherbe                                                           B  G   R  W  \
Kat-Nr ORT                              BEF       Typ                           
186    Bruxelles-Nganda (BRU; Fpl. 186) 85/101 () BKW             -  -   1  -   
188    Lokekya (LKK; Fpl. 188)          85/101 () BKW/rezent (?)  -  1   -  -   
189    Bobangi (BOB; Fpl. 189)          85/101 () ?               -  -  12  4   
                                                  Botendo         -  -  13  1   
                                                  Botendo (?)     -  1   7  1   

Scherbe                                                           All  
Kat-Nr ORT                              BEF       Typ                  
186    Bruxelles-Nganda (BRU; Fpl. 186) 85/101 () BKW               1  
188    Lokekya (LKK; Fpl. 188)          85/101 () BKW/rezent (?)    1  
189    Bobangi (BOB; Fpl. 189)          85/101 () ?                16  
                                                  Botendo          14  
                                                  Botendo (?)       9

In [142]:
df_pivot_a = df_pivot.reset_index()
display(df_pivot_a.head())

Scherbe,Kat-Nr,ORT,BEF,Typ,B,G,R,W,All
0,186,Bruxelles-Nganda (BRU; Fpl. 186),85/101 (),BKW,-,-,1,-,1
1,188,Lokekya (LKK; Fpl. 188),85/101 (),BKW/rezent (?),-,1,-,-,1
2,189,Bobangi (BOB; Fpl. 189),85/101 (),?,-,-,12,4,16
3,189,Bobangi (BOB; Fpl. 189),85/101 (),Botendo,-,-,13,1,14
4,189,Bobangi (BOB; Fpl. 189),85/101 (),Botendo (?),-,1,7,1,9


In [143]:
# um später Subtotals je multi-Index-Element zu haben, das Ganze nochmal ohne den 'Typ':
df_sum = df.pivot_table(values = 'Anzahl', index = ['Kat-Nr', 'ORT', 'BEF'], columns = 'Scherbe', aggfunc = sum, fill_value = '-', margins = True)
df_sum = df_sum.reset_index()
display(df_sum.head())

Scherbe,Kat-Nr,ORT,BEF,B,G,R,W,All
0,186,Bruxelles-Nganda (BRU; Fpl. 186),85/101 (),-,-,1,-,1
1,188,Lokekya (LKK; Fpl. 188),85/101 (),-,1,-,-,1
2,189,Bobangi (BOB; Fpl. 189),85/101 (),-,1,33,10,44
3,190,Bokwango (BKW; Fpl. 190),85/101 (),-,2,7,-,9
4,191,Zamba (ZAM; Fpl. 191),85/101 (),-,-,18,7,25


In [144]:
df_all = df_sum.append(df_pivot_a).set_index(['Kat-Nr','ORT', 'BEF', 'Typ']).sort_index()
df_all = df_all.reset_index()
df_all['Typ'] = df_all['Typ'].replace(np.nan, 'TOTAL', regex=True)
df_all['BEF'] = df_all['BEF'].str.strip('()')
df_all = df_all.drop(['Kat-Nr'], axis=1)
df_all = df_all.rename(columns={'All': 'Tot.', 'BEF':'Komplex', 'B': 'BS', 'G': 'Gef.', 'R': 'RS', 'W': 'WS'})
df_all = df_all[['ORT', 'Komplex', 'Typ', 'Gef.', 'RS', 'WS', 'BS', 'Tot.']]
df_all = df_all.set_index(['ORT', 'Komplex', 'Typ'])
df_all

Gef.  \
ORT                              Komplex            Typ                     
Bruxelles-Nganda (BRU; Fpl. 186) 85/101             TOTAL               -   
                                                    BKW                 -   
Lokekya (LKK; Fpl. 188)          85/101             TOTAL               1   
                                                    BKW/rezent (?)      1   
Bobangi (BOB; Fpl. 189)          85/101             TOTAL               1   
                                                    ?                   -   
                                                    Botendo             -   
                                                    Botendo (?)         1   
                                                    EBA/Bondongo (?)    -   
Bokwango (BKW; Fpl. 190)         85/101             TOTAL               2   
                                                    ?                   -   
                                                    BKW                 1   
                                                    BKW (?)             -   
                                                    Botendo             1   
                                                    Botendo (?)         -   
Zamba (ZAM; Fpl. 191)            85/101             TOTAL               -   
                                                    ?                   -   
                                                    Bondongo (?)        -   
                                                    Botendo (?)         -   
                                                    Mbandaka (?)        -   
Ilanga (ILA; Fpl. 192)           85/101             TOTAL               -   
                                                    ?                   -   
                                                    BKW                 -   
                                                    Botendo             -   
                                                    Botendo (?)         -   
Loka (LKA; Fpl. 193)             85/101             TOTAL               -   
                                                    ?                   -   
                                                    BKW                 -   
                                                    Bondongo            -   
                                                    Bondongo (?)        -   
...                                                                   ...   
Epena (EPE; Fpl. 303)             87/101             EBA (?)             -   
                                                    EPE                 2   
                                                    EPE (?)             -   
                                                    NGO (?)             -   
                                                    PIKMUN (?)          -   
Munda (MUN; Fpl. 304)            87/101             TOTAL               1   
                                                    EBA                 -   
                                                    EBA/EPE (?)         -   
                                                    EPE                 -   
                                                    EPE (?)             1   
                                                    EPE/EBA (?)         -   
                                                    PIKMUN (?)          -   
Itanga (ITN; Fpl. 305)           87/101 (neues Dorf TOTAL               -   
                                                    EPE                 -   
                                                    EPE (?)             -   
                                 87/103 (Elali      TOTAL               7   
                                                    ?                   -   
                                                    EBA                 -   
                                                    EPE                 7   
                                                    EPE (?)     

In [147]:
df_all.to_latex('../output/tabs/Kat-B_Tab_Oberflaechenkomplexe.tex', longtable = True, escape = False)